In [2]:
pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 5.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 20.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import numpy as np


def approx_fprime(f, x, eps=1e-8):
    
    if x.dim() != 1:
        raise ValueError("Gievn x is not 1D")

    K = x.shape[0]
    grad = torch.zeros_like(x, dtype=torch.float64)
    x = x.clone().detach().to(torch.float64)

    f_x = f(x)  

    for i in range(K):
        x1 = x.clone()
        x1[i] += eps                   
        f_x1 = f(x1)
        grad[i] = (f_x1 - f_x) / eps
    return grad


In [10]:
def test_function(x):
    return x[1]**2 + torch.exp(x[0])


In [14]:
def compare_gradients():
    eps = 1e-8
    test_points = [
        torch.tensor([0.0, 0.0], dtype=torch.float64),
        torch.tensor([1.0, 2.0], dtype=torch.float64),
        torch.tensor([-2.0, -1.0], dtype=torch.float64)
    ]

    for i, x in enumerate(test_points, 1):
        x_autograd = x.clone().detach().requires_grad_(True)
        f = test_function(x_autograd)
        f.backward()
        autodiff = x_autograd.grad.detach()
        numerical = approx_fprime(test_function, x, eps=eps)
        difference = torch.abs(numerical - autodiff)

        # Print results
        print(f"\nTest point {i}: x  = {x.numpy()}")
        print(f"  Numerical gradient = {numerical.numpy()}")
        print(f"  Autodiff gradient  = {autodiff.numpy()}")
        print(f"  |Difference|       = {difference.numpy()}")

if __name__ == "__main__":
    compare_gradients()



Test point 1: x  = [0. 0.]
  Numerical gradient = [0.99999999 0.        ]
  Autodiff gradient  = [1. 0.]
  |Difference|       = [6.07747097e-09 0.00000000e+00]

Test point 2: x  = [1. 2.]
  Numerical gradient = [2.71828187 3.99999998]
  Autodiff gradient  = [2.71828183 4.        ]
  |Difference|       = [3.78061702e-08 2.43098839e-08]

Test point 3: x  = [-2. -1.]
  Numerical gradient = [ 0.13533528 -2.00000001]
  Autodiff gradient  = [ 0.13533528 -2.        ]
  |Difference|       = [6.91768631e-09 1.00495186e-08]
